In [2]:
# use the code release version for tracking and code modifications. use the
# CHANGELOG.md file to keep track of version features, and/or release notes.
# the version file is avaiable at project root directory, check the
# global configuration setting for root directory information.
# the file is already read and is available as `__version__`
__version__ = open("../VERSION", "rt").read() # bump codecov
print(f"Current Code Version: {__version__}") # TODO : author, contact

Current Code Version: v0.1


### Code Imports

In [3]:
!pip install -r ../requirements.txt

import os     # miscellaneous os interfaces
import sys    # configuring python runtime environment
import datetime as dt
import time   # library for time manipulation, and logging
from pathlib import Path
import pickle # load/save model as a pickle file

import pandas as pd  # data manipulation and analysis
import numpy as np   # numerical operations
import torch         # pytorch library for tensor operations
import fastai


In [4]:
# from copy import deepcopy      # dataframe is mutable
# from tqdm import tqdm as TQ    # progress bar for loops
# from uuid import uuid4 as UUID # unique identifier for objs

### Data Analysis Setup

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%precision 3
%matplotlib inline
sns.set_style('whitegrid');
# plt.style.use('default-style'); # http://tinyurl.com/mpl-default-style

pd.set_option('display.max_rows', 50) # max. rows to show
pd.set_option('display.max_columns', 17) # max. cols to show
np.set_printoptions(precision = 3, threshold = 15) # set np options
pd.options.display.float_format = '{:,.3f}'.format # float precisions

In [5]:
from datetime import datetime

def to_seconds(time_obj):
    """
    Convert a time of day object into a number representing the number of seconds since the start of the day.
    
    Args:
        time_obj (datetime.time): Time of day object.
    
    Returns:
        float: Number of seconds since the start of the day.
    """
    return (time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second)


In [8]:
# import pmdarima as pm # let `auto_arima()` find the coefficients
# import statsmodels.api as sm # statsmodels for statistical generates

In [10]:
# from stationarity import checkStationarity # https://gist.github.com/ZenithClown/f99d7e1e3f4b4304dd7d43603cef344d

In [11]:
# append `src` and sub-modules to call additional files these directory are
# project specific and not to be added under environment or $PATH variable
# sys.path.append(os.path.join("..", "src", "agents")) # agents for reinforcement modelling
# sys.path.append(os.path.join("..", "src", "engine")) # derivative engines for model control
# sys.path.append(os.path.join("..", "src", "models")) # actual models for decision making tools

## Global Argument(s)

The global arguments are *notebook* specific, however they may also be extended to external libraries and functions on import. The *boilerplate* provides a basic ML directory structure which contains a directory for `data` and a separate directory for `output`. In addition, a separate directory (`data/processed`) is created to save processed dataset such that preprocessing can be avoided.

In [6]:
ROOT = ".." # the document root is one level up, that contains all code structure
DATA = Path(ROOT) / "data" # the directory contains all data files
RAW_DATA = DATA / "raw"

# processed data directory can be used, such that preprocessing steps is not
# required to run again-and-again each time on kernel restart
PROCESSED_DATA = DATA / "processed"

# OUTPUT_DIR = os.path.join(ROOT, "output")
# IMAGES_DIR = os.path.join(OUTPUT_DIR, "images")
# MODELS_DIR = os.path.join(OUTPUT_DIR, "savedmodels")

## Historic Price Data

The historic data, i.e., any data in time series format, can be read using `pd.read_*().set_index("date")` or use a custom function to read and process from the file.

In [7]:
df = pd.read_csv(PROCESSED_DATA / "time_series_data.csv", parse_dates=True)

# Convert the "timestamp" column to datetime type
df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True).dt.tz_convert("Australia/Sydney")
df["time_of_day"] = df["timestamp"].dt.time.apply(to_seconds)
df["day_of_week"] = df["timestamp"].dt.day_of_week
df["year"] = df["timestamp"].dt.year
df["month"] = df["timestamp"].dt.month

df

,date,timestamp,solar_energy_exported,generator_energy_exported,grid_energy_imported,grid_services_energy_imported,grid_services_energy_exported,grid_energy_exported_from_solar,...,total_solar_generation,total_home_usage,total_battery_discharge,total_grid_energy_exported,time_of_day,day_of_week,year,month
0,2021-04-22,2021-04-22 16:30:00+10:00,12.136,0,0.988,0,0,0.000,...,12.136,23.124,10.000,NaN,59400,3,2021,4
1,2021-04-22,2021-04-22 16:35:00+10:00,4.079,0,5.662,0,0,0.000,...,4.079,59.741,50.000,NaN,59700,3,2021,4
2,2021-04-22,2021-04-22 16:40:00+10:00,0.060,0,9.036,0,0,0.000,...,0.060,89.096,80.000,NaN,60000,3,2021,4
3,2021-04-22,2021-04-22 16:45:00+10:00,0.068,0,11.462,0,0,0.000,...,0.068,81.530,70.000,NaN,60300,3,2021,4
4,2021-04-22,2021-04-22 16:50:00+10:00,0.062,0,14.791,0,0,0.000,...,0.062,94.854,80.000,NaN,60600,3,2021,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208356,2024-09-26,2024-09-26 23:35:00+10:00,0.000,0,0.000,0,0,0.000,...,NaN,41.000,42.000,1.000,84900,3,2024,9
208357,2024-09-26,2024-09-26 23:40:00+10:00,0.000,0,1.000,0,0,0.000,...,NaN,223.000,222.000,NaN,85200,3,2024,9
208358,2024-09-26,2024-09-26 23:45:00+10:00,0.000,0,0.000,0,0,0.000,...,NaN,36.000,36.000,NaN,85500,3,2024,9
208359,2024-09-26,2024-09-26 23:50:00+10:00,0.000,0,12.000,0,0,0.000,...,NaN,58.000,46.000,NaN,85800,3,2024,9


In [8]:
# Specify the date you want to filter for
filter_date = '2024-09-25'
start_date = dt.datetime.strptime(filter_date, '%Y-%m-%d').date()
end_date = dt.datetime.strptime(filter_date, '%Y-%m-%d').date()

data = df[(df['timestamp'].dt.date >= start_date) & (df['timestamp'].dt.date <= end_date)] 

print(f"From {start_date} to {end_date}: {len(data)} rows")
data.head()

From 2024-09-25 to 2024-09-25: 288 rows


,date,timestamp,solar_energy_exported,generator_energy_exported,grid_energy_imported,grid_services_energy_imported,grid_services_energy_exported,grid_energy_exported_from_solar,...,total_solar_generation,total_home_usage,total_battery_discharge,total_grid_energy_exported,time_of_day,day_of_week,year,month
207785,2024-09-25,2024-09-25 00:00:00+10:00,0.000,0,40.000,0,0,0.000,...,NaN,40.000,NaN,NaN,0,2,2024,9
207786,2024-09-25,2024-09-25 00:05:00+10:00,0.000,0,158.000,0,0,0.000,...,NaN,40.000,NaN,NaN,300,2,2024,9
207787,2024-09-25,2024-09-25 00:10:00+10:00,0.000,0,86.000,0,0,0.000,...,NaN,44.000,NaN,NaN,600,2,2024,9
207788,2024-09-25,2024-09-25 00:15:00+10:00,0.000,0,95.000,0,0,0.000,...,NaN,95.000,NaN,NaN,900,2,2024,9
207789,2024-09-25,2024-09-25 00:20:00+10:00,0.000,0,80.000,0,0,0.000,...,NaN,80.000,NaN,NaN,1200,2,2024,9


In [9]:
FEATURE_COLUMNS = [
    "time_of_day",
    "day_of_week",
    "month",
    "year",
    # "temperature",
    # "humidity",
    # "wind_speed",
    # "precipitation",
    # "cloud_cover",
    # "weather_description",
    "total_home_usage",
    # "grid_energy_export",
]

data = data[FEATURE_COLUMNS]
data


,time_of_day,day_of_week,month,year,total_home_usage
207785,0,2,9,2024,40.000
207786,300,2,9,2024,40.000
207787,600,2,9,2024,44.000
207788,900,2,9,2024,95.000
207789,1200,2,9,2024,80.000
...,...,...,...,...,...
208068,84900,2,9,2024,38.000
208069,85200,2,9,2024,32.000
208070,85500,2,9,2024,48.000
208071,85800,2,9,2024,30.000


## The Model

In [10]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from fastai.basics import Learner
from fastai.callback.all import *
from torch.optim import Adam
from torch.utils.data import Dataset
import matplotlib.pyplot as plt


class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out


# Load and preprocess data
def load_data(df):
    # Assuming the DataFrame has FEATURE_COLUMNS
    data = df[FEATURE_COLUMNS].values

    # Normalize data using MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    normalized_data = scaler.fit_transform(data)

    return torch.FloatTensor(normalized_data, device="cpu"), scaler


# Create sequences for LSTM
def create_sequences(data, sequence_length):
    sequences = []
    labels = []
    for i in range(len(data) - sequence_length):
        seq = data[i : i + sequence_length]
        label = data[i + sequence_length, 0]  # Predict energy consumption
        sequences.append(seq)
        labels.append(label)
    return torch.stack(sequences), torch.FloatTensor(labels)


# Create a custom Dataset for fastai
class TimeSeriesDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

In [32]:
from functools import partial
from sklearn.metrics import mean_squared_error

# Hyperparameters

input_size = len(FEATURE_COLUMNS)
hidden_size = 50  # LSTM hidden layer size
num_layers = 2  # Number of LSTM layers
output_size = 1  # Predicting one value (Energy consumption)
sequence_length = 24  # Sequence length
batch_size = 64  # Batch size for training
num_epochs = 10  # Number of training epochs
learning_rate = 0.001


# Load and preprocess data
input_data, scaler = load_data(data)
sequences, labels = create_sequences(input_data, sequence_length)

# Create DataLoader using fastai
dataset = TimeSeriesDataset(sequences, labels)

# train_size = int(0.8 * len(dataset))
# valid_size = len(dataset) - train_size
# train_ds, valid_ds = torch.utils.data.random_split(dataset, [train_size, valid_size])

trainloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)
# validloader = torch.utils.data.DataLoader(valid_ds, batch_size=4, shuffle=False)

# Define model, loss function, and optimizer
# device = torch.device(
#     "cuda"
#     if torch.cuda.is_available()
#     else "mps" if torch.backends.mps.is_available() else "cpu"
# )
device = "cpu"
model = LSTM(input_size, hidden_size, num_layers, output_size).to(device)
criterion = nn.MSELoss()

opt_func = partial(fastai.optimizer.OptimWrapper, opt=Adam)

# Define fastai Learner
learn = Learner(
    dls=fastai.data.core.DataLoaders(trainloader, trainloader),
    model=model,
    loss_func=criterion,
    opt_func=opt_func,
    metrics=[mean_squared_error],
).to_fp32()

In [33]:

# Train the model using fastai's training loop
learn.fit_one_cycle(num_epochs)


epoch,train_loss,valid_loss,mean_squared_error,time


/Users/eho/dev/energy-consumption-prediction/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


TypeError: Exception occured in `Recorder` when calling event `after_batch`:
	can't convert mps:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
plt.plot(data["values"], label = "Historic Price", color = "#5d8bd4")
plt.plot(data["values"].ewm(alpha = alpha, adjust = False).mean(), label = f"Adjusted EWMA (span = {span}, α = {alpha})", color = "#b09666")

plt.legend(loc = "upper right")
plt.title(commodity)
plt.show()

## Model Development

An exploratory data analysis on a time series data set is typically different from that of conventional/non-timeseries dataset. The [`template`](https://github.com/ZenithClown/ai-ml-project-template) provides some basic analysis techniques that is applicable to any type of datasets. The following methods are implemented:

  * [**Seasonal Decomposition**](https://machinelearningmastery.com/decompose-time-series-data-trend-seasonality/): The process of breaking a time series data into three main components - (I) trend, (II) seasonality, and (III) residuals/error terms.
  * [**Check Data Stationarity**](https://www.analyticsvidhya.com/blog/2021/06/statistical-tests-to-check-stationarity-in-time-series-part-1/) Basic statistical models like `ARIMA` works on the principle that the time series is stationary.

But, first we copy the actual dataset into a copied variable (preserve original data), and understand the data to be applied on to the master series. In addition, we also set the date time indexing (with frequency, if not already available) since may functions (like ETS Decomposition) is dependent on it.

In [ ]:
frame_ = data[["values"]].copy() # frame with missing dates, and imputed with frequency
all_dates = list(map(lambda x : pd.Timestamp(str(x)), list(dt_.date_range(frame_.index.min().date(), frame_.index.max().date()))[::7])) # weekly frequency data
missing_dates = [date for date in all_dates if date not in frame_.index.values] # ? this function is not authorized

# insert missing dates
frame_.reset_index(inplace = True)
for missing_date in missing_dates:
    frame_.loc[len(frame_)] = [missing_date, pd.NA]
    
# interpolate on missing dates
frame_ = frame_.sort_values("date").set_index("date") # don't sort descending
frame_["values"] = frame_["values"].interpolate() # fill missing values
frame_ = frame_.resample("???").first() # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html

In [ ]:
plt.figure(figsize = (25, 19))
seasonal_decompose(frame_).plot();
plt.show()

In [ ]:
# get yearly average sales
yearly_sales = data.resample("Y")[commodity].mean().reset_index()

# get monthly average sales, grouped by year
monthly_sales = data.resample("M")[commodity].mean().reset_index()
monthly_sales["year"] = monthly_sales["date"].dt.year # can be used as an hue parameter to distinguish

plt.figure(figsize = (25, 5))
sns.lineplot(yearly_sales, x = "date", y = commodity, label = "Yearly Average Sales")
sns.lineplot(monthly_sales, x = "date", y = commodity, hue = "year", palette = "viridis", label = "Monthly Average Sales")

# disable/set xy label
plt.xlabel("")
plt.ylabel("Price Values")

plt.legend([]) # yearly diff color is understood
plt.title("Average Sales Historic Price Trend")

plt.show()

In [ ]:
window = 12
*_, rolling = checkStationarity(data, feature = "values", window = window)

plt.plot(rolling)
plt.suptitle("Rolling Mean & Standard Deviation")
plt.title(f"Rolling Window Size = {window}")
plt.show()